<a id="top"></a>

# OpenVINO Implementation Exercise

# Introduction

This exercise purpose is oriented towards showing you a demonstration on how OpenVINO works. It is a modifed version of the Classification Tutorial made by Intel. You can find it under the OpenVINO sample in the DevCloud.

# Steps to cover:

## A. Importing Main Libraries

## B. Creating useful functions  
#### 1. loadInputImage
#### 2. resizeInputImage
#### 3. showImage
#### 4. processAndDisplayResults
#### 5. inferImage
#### 6. batchLoadInputImages
#### 7. batchProcessAndDisplayResults

## C. Setting up OpenVINO 
#### I.     Getting the pre-traine model
#### II.    Using the Model Optimizer
#### III.    Configuring OpenVINO parameters
#### IV.     Creating the inference engine instance
#### V.     Creating IENetwork & loading the model in it
#### VI.   Checking layer compatibility
#### VII. Loading the model into the plugin
#### VIII.     Loading labels
#### IX.     Preparing the Inputs
#### X. Running the Inference
#### XI.     Processing and Displaying Results

## D. Additional Info
#### a. Loading a different image
#### b. Loading an image from the internet
#### c. Using multiple images
#### d. Running multiple images
#### e. Processing & displaying multiple images


# Exercise time
## 01. Load and Process 10 AI generated images
## 10. Change the display format
## 11. Change the following

## A. Importing Main Libraries 
Before we start coding, we will export the libraries necessary for the model implementaton to work

In [ ]:
import os
import time 
import cv2
import numpy as np
from openvino.inference_engine import IECore
%matplotlib inline 
from matplotlib import pyplot as plt 

## B. Creating useful functions  
Before we go into using setting up and running our model, we'll create some functions that will help us our format everything

### 1.      loadInputImage

In [ ]:
def loadInputImage(input_path):
    # globals to store input width and height
    global input_w, input_h

    # use OpenCV to load the input image
    cap = cv2.VideoCapture(input_path)

    # store input width and height
    input_w = cap.get(3)
    input_h = cap.get(4)
    print("Loaded input image [",input_path,"]","\n",
          "resolution=", input_w,"w x ", input_h, "h",)

    # load the input image
    ret, image = cap.read()
    del cap
    return image

### 2.      resizeInputImage

In [ ]:
def resizeInputImage(image):
    # resize image dimensions form image to model's input w x h
    in_frame = cv2.resize(image, (w, h))
    # Change data layout from HWC to CHW
    in_frame = in_frame.transpose((2, 0, 1))
    # reshape to input dimensions
    in_frame = in_frame.reshape((n, c, h, w))
    print(f"Resized input image from {image.shape[:-1]} to {(h, w)}")
    return in_frame

### 3.      showImage

In [ ]:
def showImage(image_name,orig_input_image):
        # display image
    print("↑ RESULTS FOR THE", image_name, "CAN BE FINDED AT THE TOP ↑")
    plt.figure()
    plt.axis("off")
    im_to_show = cv2.cvtColor(orig_input_image, cv2.COLOR_BGR2RGB)
    # show input image
    plt.imshow(im_to_show)

### 4.      processAndDisplayResults

In [ ]:
def processAndDisplayResults(probs, orig_input_image, orig_input_path, image_name):
    
    # report top n results for image
    print("Top ", report_top_n, " results for image", orig_input_path, ":")

    # remove dimensions of length=1
    probs = np.squeeze(probs)

    # sort then return top report_top_n entries
    top_ind = np.argsort(probs)[-report_top_n:][::-1]
    
    # print out top probabilities, looking up label
    print("Probability% is <label>")
    for id in top_ind:
        det_label = labels_map[id] if labels_map else f"#{id}"
        print(f" {probs[id]*100:.7f} % is {det_label}")
    print("_______________________________________________________")
    
    showImage(image_name,orig_input_image)

### 5.      inferImage

In [ ]:
def inferImage(image, input_path, image_name):
    # prepare input
    in_frame = resizeInputImage(image)

    # run inference
    res = exec_net.infer(inputs={input_blob: in_frame})

    # process inference results
    processAndDisplayResults(res[output_blob][0], image, input_path,image_name)

### 6.      batchLoadInputImages

In [ ]:
# define function to load input images into input batch
def batchLoadInputImages(batch_paths):
    global batch_size
    global batch_images
    global orig_image_paths
    global orig_images
    batch_size = len(batch_paths)

    # create input batch (array) of input images
    batch_images = np.ndarray(shape=(batch_size, c, h, w))

    # create array to hold original images and paths for displaying later
    orig_images = []
    orig_image_paths = []

    for i in range(batch_size):
        # load image
        image = loadInputImage(batch_paths[i])

        # save original image and path
        orig_images.append(image)
        orig_image_paths.append(batch_paths[i])

        # prepare input
        in_frame = resizeInputImage(image)

        # add input to batch
        batch_images[i] = in_frame
    return batch_size, batch_images, orig_image_paths, orig_images

### 7.      batchProcessAndDisplayResults

In [ ]:
# create function to process inference results
def batchProcessAndDisplayResults(result, orig_input_images, orig_image_paths, orig_image_name):
    # get output results
    res = result[output_blob]

    for i, probs in enumerate(res):
        processAndDisplayResults(probs, orig_input_images[i], orig_image_paths[i], orig_image_name[i])

## C. Creating useful functions  


### I.     Getting the pre-traine model


In [ ]:
!downloader.py --name squeezenet1.1 -o raw_model

### II.    Using the Model Optimizer

In [ ]:
!converter.py --name squeezenet1.1 -d raw_model -o model

### III.    Configuring OpenVINO parameters

In [ ]:
# model IR files
model_xml = "model/public/squeezenet1.1/FP32/squeezenet1.1.xml"
model_bin = "model/public/squeezenet1.1/FP32/squeezenet1.1.bin"

# input image file
input_path = "./ai1_dogo.jpg"

# CPU extension library to use
cpu_extension_path = (
    os.path.expanduser("~")
    + "/inference_engine_samples/intel64/Release/lib/libcpu_extension.so"
)

# device to use
device = "CPU"

# number of top results to display
report_top_n = 10

# output labels
labels_path = "squeezenet1.1.labels"

print( "Configuration parameters settings:", 
       "\n\tmodel_xml=", model_xml,
       "\n\tmodel_bin=", model_bin,
       "\n\tinput_path=", input_path,
       "\n\tdevice=", device,
       "\n\tlabels_path=", labels_path,
       "\n\treport_top_n=",report_top_n,)

### IV.     Creating the inference engine instance

In [ ]:
# create Inference Engine instance
ie = IECore()
print("An Inference Engine object has been created")

### V.     Creating the IENetwork & loading the model in it

In [ ]:
# load network from IR files
net = ie.read_network(model=model_xml, weights=model_bin)
print("Loaded model IR files [", model_bin, "] and [", model_xml, "]\n")

### VI.   Checking layer compatibility

In [ ]:
# check to make sure that the plugin has support for all layers in the loaded model
supported_layers = ie.query_network(net, device)
not_supported_layers = [l for l in net.layers.keys() if l not in supported_layers]
if len(not_supported_layers) != 0:
    print("ERROR: Following layers are not supported by the plugin for specified",
          " device {}:\n {}".format(device, ", ".join(not_supported_layers)),)
    if 0 != 1:
        raise Exception("ERROR: Missing support for all layers in the model, cannot continue.")

    # check to make sue that the model's input and output are what is expected
    if len(net.inputs.keys()) != 1:
        raise Exception("ERROR: This sample supports only single input topologies.")
    if len(net.outputs) != 1:
        raise Exception("ERROR: This sample supports only single output topologies.")
print("SUCCESS: Model IR files have been loaded and verified")

### VII. Loading the model into the plugin

In [ ]:
exec_net = ie.load_network(network=net, num_requests=2, device_name=device)

# store name of input and output blobs
input_blob = next(iter(net.inputs))
output_blob = next(iter(net.outputs))

# read the input's dimensions: n=batch size, c=number of channels, h=height, w=width
n, c, h, w = net.inputs[input_blob].shape
print("Loaded model into Inference Engine for device:", device,
      "\nModel input dimensions:",
      " n=",n,
      ", c=",c,
      ", h=",h,
      ", w=",w,)

### VIII.     Loading labels

In [ ]:
labels_map = None
# if labels points to a label mapping file, then load the file into labels_map
print(labels_path)
if os.path.isfile(labels_path):
    with open(labels_path) as f:
        labels_map = [x.split(sep=" ", maxsplit=1)[-1].strip() for x in f]
    print("Loaded label mapping file [", labels_path, "]")
else:
    print("No label mapping file has been loaded, only numbers will be used",
          " for detected object labels",)

### IX.     Preparing the Inputs

In [ ]:
# load image
image = loadInputImage(input_path)

# resize the input image
in_frame = resizeInputImage(image)

# display input image
plt.axis("off")
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
print("The AI dogo:")

### X.     Running the Inference

In [ ]:
# save start time
inf_start = time.time()

# run inference
res = exec_net.infer(inputs={input_blob: in_frame})
# calculate time from start until now
inf_time = time.time() - inf_start
print(f"Inference complete, run time: {inf_time * 1000:.3f} ms")

### XI.     Processing and Displaying Results

In [ ]:
processAndDisplayResults(res[output_blob][0], image, input_path,"AI dogo")

## D. Exercise explanation

### a. Loading a different image

In [ ]:
# set path to different input image
input_path = "ai1_cat.jpg"

# load input image
image = loadInputImage(input_path)

# infer image and display results
inferImage(image, input_path,"AI Cat")

### b. Loading an image from the internet

In [ ]:
# input_path may be set to a local file or URL
input_path = "https://upload.wikimedia.org/wikipedia/commons/5/56/Sea_Anemone_%2810062178943%29.jpg"

# load input image
image = loadInputImage(input_path)

# infer image and display results
inferImage(image, input_path, "Anemone")

### c. Loading multiple images

In [ ]:
# batch of inputs which may be local files or URLs (comma separated)
batch_paths = ["./ai2_dogo.jpg", "./ai1_cat.jpg", "./ai1_bird.jpg"]
batch_image_names = ["AI Dogo 2", "AI Cat", "AI Bird"]
batchLoadInputImages(batch_paths)
print("\nLoaded", batch_size, "images.")

### d. Running multiple images

In [ ]:
# set the batch size to match the number of input images
net.batch_size = batch_size
print("Network batch size set to", batch_size)

## reload network because batch size has changed
exec_net = ie.load_network(network=net, num_requests=2, device_name=device)


# save start time
inf_start = time.time()

# run inference
res = exec_net.infer(inputs={input_blob: batch_images})

# calculate time from start until now
inf_time = time.time() - inf_start
print(f"Inference complete, run time: {inf_time * 1000:.3f} ms")

### e. Processing & displaying multiple images

In [ ]:
# process inference results
batchProcessAndDisplayResults(res, orig_images, orig_image_paths, batch_image_names)

# Exercise time

## 01. Load and Process 10 AI generated images

For your first task, you will use the already setted up model, and use to analyze 10 new images. You can use the internet or a generative AI website to do this.

In [ ]:
# batch of inputs which may be local files or URLs (comma separated)
batch_paths = [ "https://cdn.pixabay.com/photo/2015/06/25/12/27/daisy-821222_1280.jpg", "./cat.jpg", ]

# load batch of inputs
batchLoadInputImages(batch_paths)
print("Loaded", batch_size, "images.")

# set the batch size to match the number of input images
net.batch_size = batch_size
print("Network batch size set to", batch_size)

# reload network because batch size has changed
exec_net = ie.load_network(network=net, num_requests=2, device_name=device)

# run inference
res = exec_net.infer(inputs={input_blob: batch_images})

# process inference results
batchProcessAndDisplayResults(res, orig_images, orig_image_paths)

print("Done.")

## 10. Change the display format

Your second task is using a different way to display the results of the image analysis. You can modify some of the printed, or you can also use Matplotlib to display a graph of the results.

In [ ]:
def processAndDisplayResults(probs, orig_input_image, orig_input_path, image_name):
    
    # report top n results for image
    print("Top ", report_top_n, " results for image", orig_input_path, ":")

    # remove dimensions of length=1
    probs = np.squeeze(probs)

    # sort then return top report_top_n entries
    top_ind = np.argsort(probs)[-report_top_n:][::-1]
    
    # print out top probabilities, looking up label
    print("Probability% is <label>")
    for id in top_ind:
        det_label = labels_map[id] if labels_map else f"#{id}"
        print(f" {probs[id]*100:.7f} % is {det_label}")
    print("_______________________________________________________")
    
    showImage(image_name,orig_input_image)

In [ ]:
# Prepare your inputs - CODE GOES BELLOW 


In [ ]:
# Running the Inference - CODE GOES BELLOW 


In [ ]:
# Process and Display your results - CODE GOES BELLOW


## 11. Change the following
The last task is modifying our previous setup. You'll need to:"\n"

1. Change the number of posible things in the analysis
2. Change the device used to compute the model
3. Change the model from 1.1 to 1.0

In [4]:
# Go Nuts